In [1]:
import os,cv2
import numpy as np
faceDetectClassifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

In [2]:
PATH = os.getcwd()
# Define data path
labels_name={}
data_path = PATH + '/dataset'
data_dir_list = os.listdir(data_path)
for i in range(0,len(data_dir_list)):
    labels_name[data_dir_list[i]]=i
labels_name

{'Penny': 0, 'Sheldon': 1}

In [3]:
img_data_list=[]
labels_list = []

for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    print ('Loading the images of dataset-'+'{}\n'.format(dataset))
    label = labels_name[dataset]
    i=0
    for img in img_list:
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img )
        input_img=cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
        input_img_resize=cv2.resize(input_img,(128,128))
        img_data_list.append(input_img_resize)
        labels_list.append(label)
    

Loading the images of dataset-Penny

Loading the images of dataset-Sheldon



In [4]:
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data /= 255
print (img_data.shape)
num_classes=len(data_dir_list)
labels = np.array(labels_list)

(668, 128, 128)


In [5]:
from keras import backend as K
K.set_image_dim_ordering('th')
num_channel=1
if num_channel==1:
	if K.image_dim_ordering()=='th':
		img_data= np.expand_dims(img_data, axis=1) 
		print (img_data.shape)
	else:
		img_data= np.expand_dims(img_data, axis=4) 
		print (img_data.shape,"LLLLLL")
		
else:
	if K.image_dim_ordering()=='th':
		img_data=np.rollaxis(img_data,3,1)
		print (img_data.shape)

C:\Users\UTKARSH\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


(668, 1, 128, 128)


In [6]:
from keras.utils import np_utils
print(np.unique(labels,return_counts=True))
Y = np_utils.to_categorical(labels, num_classes)

(array([0, 1]), array([251, 417], dtype=int64))


In [7]:
from sklearn.cross_validation import train_test_split
x,y = img_data,Y
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

C:\Users\UTKARSH\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [8]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D,Conv2D
from keras.optimizers import SGD,RMSprop,adam

In [9]:
input_shape=img_data[0].shape
print(img_data[0].shape)
model = Sequential()
model.add(Convolution2D(32, 3,3,border_mode='same',input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop',metrics=["accuracy"])

(1, 128, 128)


C:\Users\UTKARSH\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(1, 128, 1..., padding="same")`
  after removing the cwd from sys.path.
C:\Users\UTKARSH\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  
C:\Users\UTKARSH\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
  # This is added back by InteractiveShellApp.init_path()


In [10]:
# Viewing model_configuration

model.summary()
model.get_config()
model.layers[0].get_config()
model.layers[0].input_shape			
model.layers[0].output_shape			
model.layers[0].get_weights()
np.shape(model.layers[0].get_weights()[0])
model.layers[0].trainable

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 128, 128)      320       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 128, 128)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 126, 126)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 126, 126)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 63, 63)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 63, 63)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 61, 61)        18496     
__________

True

In [ ]:
hist = model.fit(X_train, y_train, batch_size=16,nb_epoch=10 , verbose=1, validation_data=(X_test, y_test))

C:\Users\UTKARSH\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Train on 534 samples, validate on 134 samples
Epoch 1/10
534/534 [==============================] - 106s 199ms/step - loss: 3.8348 - acc: 0.6386 - val_loss: 0.5909 - val_acc: 0.9776
Epoch 2/10
534/534 [==============================] - 104s 194ms/step - loss: 0.1650 - acc: 0.9401 - val_loss: 0.0318 - val_acc: 1.0000
Epoch 3/10
534/534 [==============================] - 104s 195ms/step - loss: 0.0437 - acc: 0.9888 - val_loss: 0.0099 - val_acc: 1.0000
Epoch 4/10
534/534 [==============================] - 104s 195ms/step - loss: 0.0167 - acc: 0.9944 - val_loss: 0.0154 - val_acc: 1.0000
Epoch 5/10
432/534 [=======================>......] - ETA: 17s - loss: 0.0212 - acc: 0.9931

In [12]:
# Evaluating the model

score = model.evaluate(X_test, y_test, verbose=0)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

test_image = X_test[0:1]
print (test_image.shape)

print(model.predict(test_image))
print(model.predict_classes(test_image))
print(y_test[0:1])

Test Loss: 0.0015602091209156745
Test accuracy: 1.0
(1, 1, 128, 128)
[[4.5981047e-19 1.0000000e+00]]
[1]
[[0. 1.]]


In [13]:
# Testing a new image
test_image1 = cv2.imread('test//penny.jpg')
test_image1=cv2.cvtColor(test_image1, cv2.COLOR_BGR2GRAY)
test_image1=cv2.resize(test_image1,(128,128))
print (test_image1.shape)
test_image2 = cv2.imread('test//sheldon.jpg')
test_image2=cv2.cvtColor(test_image2, cv2.COLOR_BGR2GRAY)
test_image2=cv2.resize(test_image2,(128,128))
print (test_image2.shape)
t_im_list = []
t_im_list.append(test_image2)
test_data1 = np.array(t_im_list)
test_data1 = test_data1.astype('float32')
test_data1 /= 255
print (test_data1.shape)

(128, 128)
(128, 128)
(1, 128, 128)


In [14]:
flag=True

In [15]:
test_image_list = []
test_image_list.append(test_image2)
test_image_list.append(test_image1)
test_data = np.array(test_image_list)
test_data = test_data.astype('float32')
test_data /= 255
if flag:
    test_data= np.expand_dims(test_data, axis=1)
    flag=False

print (test_data.shape)

(2, 1, 128, 128)


In [16]:
print(model.predict(test_data))
key_list = list(labels_name.keys()) 
val_list = list(labels_name.values()) 
val_test=model.predict_classes(test_data)
lab_test=[]
for i in val_test:
    lab_test.append(key_list[val_list.index(i)])
print(lab_test)

[[1.1181108e-16 1.0000000e+00]
 [1.0000000e+00 5.3757732e-14]]
['Sheldon', 'Penny']


In [17]:
import cv2
import numpy as np
print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe("dependencies//deploy.prototxt.txt", "dependencies//res10_300x300_ssd_iter_140000.caffemodel")
print("[INFO] starting video stream...")
cap = cv2.VideoCapture('facevideos//tbbt.mp4')

[INFO] loading model...
[INFO] starting video stream...


In [18]:
while True:
    ret,frame = cap.read()
    if not ret:
        break
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0,(300, 300), (104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        a_type=[]
        if confidence < 0.5:
            continue
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        cropped = frame[startY:endY, startX:endX]
        cropped=cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
        cropped=cv2.resize(cropped, (128, 128))
        a_type.append(cropped)
        a_type=np.array(a_type)
        a_type=np.expand_dims(a_type, axis=1)
        #text = "{:.2f}%".format(confidence * 100)
        text=model.predict_classes(a_type)
        conff=model.predict_proba(a_type)
        print(text,conff)
        text=key_list[val_list.index(text)]
        y = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.rectangle(frame, (startX, startY), (endX, endY),(0, 0, 255), 2)
        cv2.putText(frame, text, (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
cv2.destroyAllWindows()

[1] [[0. 1.]]
[0] [[1. 0.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[0] [[1. 0.]]
[1] [[0. 1.]]
[0] [[1. 0.]]
[1] [[0. 1.]]
[0] [[1. 0.]]
[1] [[0. 1.]]
[0] [[1. 0.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[0] [[1. 0.]]
[1] [[0. 1.]]
[0] [[1. 0.]]
[1] [[0. 1.]]
[0] [[1. 0.]]
[1] [[0. 1.]]
[0] [[1. 0.]]
[1] [[0. 1.]]
[0] [[1. 0.]]
[1] [[0. 1.]]
[0] [[1. 0.]]
[1] [[0. 1.]]
[0] [[1. 0.]]
[1] [[0. 1.]]
[0] [[1. 0.]]
[1] [[0. 1.]]
[0] [[1. 0.]]
[1] [[0. 1.]]
[0] [[1. 0.]]
[1] [[0. 1.]]
[0] [[1. 0.]]
[1] [[0. 1.]]
[0] [[1. 0.]]
[1] [[0. 1.]]
[0] [[1. 0.]]
[1] [[0. 1.]]
[0] [[1. 0.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[0] [[1. 0.]]
[1] [[0. 1.]]
[0] [[1. 0.]]
[1] [[0. 1.]]
[0] [[

[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[0. 1.]]
[1] [[